<a href="https://colab.research.google.com/github/amirmahdiKhosravi/end-to-end-MLOps-project/blob/main/End_toEnd_MLOps_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup for our project:

In [8]:
%pip install "zenml[server]"
!zenml integration install sklearn -y
%pip install pyparsing==2.4.2

import IPython

IPython.Application.instance().kernel.do_shutdown(restart=True)

NumExpr defaulting to 2 threads.
⠋ Installing integrations...


{'status': 'ok', 'restart': True}

In [1]:
NGROK_TOKEN = "2Ze6COEBPZorYjW9mcX2NxGXkd0_5VjfzGigf1Q8wPQywZAmr"

In [2]:
from zenml.environment import Environment

if Environment.in_google_colab():
  !pip install pyngrok
  !ngrok authtoken {NGROK_TOKEN}

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


NumExpr defaulting to 2 threads.
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
!pip install zenml

ZenML setup:

In [4]:
!rm -rf .zen
!zenml init

NumExpr defaulting to 2 threads.
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
⠇ Initializing ZenML repository at /content.
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
Setting the repo active workspace to 'default'.
Setting the repo active stack to default.
⠹ Initializing ZenML repository at /content.
ZenML repository initialized at /content.
⠹ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.

The local active stack was initialized to 'default'. This local confi

Example experimentation ML code:

In [5]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

def train_test():
  digits = load_digits()
  data = digits.images.reshape((len(digits.images), -1))
  X_train, X_test, y_train, y_test = train_test_split(
      data, digits.target, test_size=0.2, shuffle=False
  )
  model = SVC(gamma=0.001)
  model.fit(X_train, y_train)
  test_acc = model.score(X_test, y_test)
  print(f"model accuracy: {test_acc}")

train_test()

model accuracy: 0.9583333333333334


Now we are to turn this simple experiment into ML pipeline with ZenML. this pipeline will automate the process of loading the data, training a model and evaluating it.

In [11]:
from zenml import step
from typing_extensions import Annotated
import pandas as pd
from typing import Tuple

@step
def importer() -> Tuple[
    Annotated[np.ndarray, "X_train"],
    Annotated[np.ndarray, "X_test"],
    Annotated[np.ndarray, "y_train"],
    Annotated[np.ndarray, "x_test"]
]:

  digits = load_digits()
  data = digits.images.reshape((len(digits.images), -1))
  X_train, X_test, y_train, y_test = train_test_split(data, digits.target, test_size=0.2, shuffle = False)
  return X_train, X_test, y_train, y_test

@step
def SVC_trainer(X_train: np.ndarray, y_train : np.ndarray) -> ClassifierMixin:
  model = SVC(gamma=0.001)
  model.fit(X_train, y_train)
  return model

@step
def evaluator(X_test: np.ndarray, y_test : np.ndarray, model : ClassifierMixin) -> float:
  model_acc = model.score(X_test, y_test)
  print(f"Model accuracy: {model_acc}")
  return model_acc

Creating zenML pipeline:

In [12]:
from zenml import pipeline

@pipeline
def digits_pipeline():
  X_train, X_test, y_train, y_test = importer()
  model = SVC_trainer(X_train, y_train)
  evaluator(X_test, y_test, model)

Running zenML pipeline:

In [13]:
digits_SVC_pipeline = digits_pipeline()
#digits_SVC_pipeline.run(unlisted=True)

Initiating a new run for the pipeline: digits_pipeline.
Reusing registered version: (version: 1).
Executing a new run.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
Using cached version of importer.
Step importer has started.
Using cached version of SVC_trainer.
Linking artifact output to model None version None implicitly.
Step SVC_trainer has started.
Using cached version of evaluator.
Linking artifact output to model None version None implicitly.
Step evaluator has started.
Run digits_pipeline-2023_12_17-22_05_39_817528 has finished in 0.441s.
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.


now with following code we can visualize zenML the pipeline in zenML dashboard. the username is loca and the password should be left empty.

In [17]:
from zenml.environment import Environment

def start_zenml_dashboard(port=8237):
  if Environment.in_google_colab():
    from pyngrok import ngrok

    public_url = ngrok.connect(port)
    print(f"use this url instead: {public_url}")
    !zenml up --blocking --port {port}

  else:
    !zenml up --port {port}

start_zenml_dashboard()

NumExpr defaulting to 2 threads.
Updating the local ZenML server.
⠋ Stopping service 'LocalZenServer[9a4ac2eb-dad5-4747-8f30-5e622a95d211] (type: zen_server, flavor: 
local)'.
⠙ Stopping service 'LocalZenServer[9a4ac2eb-dad5-4747-8f30-5e622a95d211] (type: zen_server, flavor: 
local)'.
⠹ Stopping service 'LocalZenServer[9a4ac2eb-dad5-4747-8f30-5e622a95d211] (type: zen_server, flavor: 
local)'.
⠸ Stopping service 'LocalZenServer[9a4ac2eb-dad5-4747-8f30-5e622a95d211] (type: zen_server, flavor: 
local)'.
⠼ Stopping service 'LocalZenServer[9a4ac2eb-dad5-4747-8f30-5e622a95d211] (type: zen_server, flavor: 
local)'.
⠴ Stopping service 'LocalZenServer[9a4ac2eb-dad5-4747-8f30-5e622a95d211] (type: zen_server, flavor: 
local)'.
⠦ Stopping service 'LocalZenServer[9a4ac2eb-dad5-4747-8f30-5e622a95d211] (type: zen_server, flavor: 
local)'.
⠧ Stopping service 'LocalZenServer[9a4ac2eb-dad5-4747-8f30-5e622a95d211] (type: zen_server, flavor: 
local)'.
⠇ Stopping service 'LocalZenServer[9a4ac2eb-dad5-4747-